In [1]:
import os, sys, time
from tqdm.notebook import tqdm
import pandas as pd
import json
import numpy as np
from collections import defaultdict
from scipy import stats
from datasets import load_from_disk
from transformers import DefaultDataCollator, ViTForImageClassification, Trainer
from utils.helper import get_device
from utils.vit_util import transforms, transforms_c100
from utils.constant import ViTExperiment
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")
import torch
import math


ds_name = "c100"
k = 0

2024-09-23 02:57:14.931628: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-23 02:57:15.087173: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-09-23 02:57:15.087215: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-09-23 02:57:15.953400: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
# デバイス (cuda, or cpu) の取得
device = get_device()
# datasetをロード (初回の読み込みだけやや時間かかる)
dataset_dir = ViTExperiment.DATASET_DIR
# バッチごとの処理のためのdata_collator
data_collator = DefaultDataCollator()
# オリジナルのds name
ds_ori_name = ds_name.rstrip("c") if ds_name.endswith("c") else ds_name
# foldが記載されたdir name
ds_dirname = f"{ds_ori_name}_fold{k}"
# dsのload
ds = load_from_disk(os.path.join(dataset_dir, ds_dirname))
ds_preprocessed = ds.with_transform(transforms)

# datasetごとに違う変数のセット
if ds_name == "c10" or ds_name == "c10c":
    tf_func = transforms
    label_col = "label"
elif ds_name == "c100" or ds_name == "c100c":
    tf_func = transforms_c100
    label_col = "fine_label"
else:
    NotImplementedError
# 読み込まれた時にリアルタイムで前処理を適用するようにする
ds_preprocessed = ds.with_transform(tf_func)
# ラベルを示す文字列のlist
labels = ds_preprocessed["train"].features[label_col].names

# pretrained modelのロード
ori_pretrained_dir = getattr(ViTExperiment, ds_ori_name).OUTPUT_DIR.format(k=k)
pretrained_dir = os.path.join(ori_pretrained_dir, "repairing_by_de")
# このpythonのファイル名を取得

model = ViTForImageClassification.from_pretrained(ori_pretrained_dir).to(device)
model.eval()
end_li = model.vit.config.num_hidden_layers
# 学習時の設定をロード
training_args = torch.load(os.path.join(ori_pretrained_dir, "training_args.bin"))
# Trainerオブジェクトの作成
training_args_dict = training_args.to_dict()
train_batch_size = training_args_dict["per_device_train_batch_size"]
eval_batch_size = training_args_dict["per_device_eval_batch_size"]
# 予測結果格納ディレクトリ
pred_out_dir = os.path.join(pretrained_dir, "pred_results", "PredictionOutput")
# pred_out_dirがなければ作成
if not os.path.exists(pred_out_dir):
    os.makedirs(pred_out_dir)
tgt_layer = 11
tgt_pos = ViTExperiment.CLS_IDX

train_iter = math.ceil(len(ds_preprocessed["train"]) / train_batch_size)
repair_iter = math.ceil(len(ds_preprocessed["repair"]) / eval_batch_size)
test_iter = math.ceil(len(ds_preprocessed["test"]) / eval_batch_size)

Device: cuda


In [3]:
for split, batch_size in zip(["repair", "test"], [eval_batch_size, eval_batch_size]):
    print(f"Process {split} (batch_size={batch_size})...")
    all_proba = []
    all_pred_labels = []
    all_before_intermediates = []
    all_intermediates = []
    all_after_intermediates = []
    # 各バッチに対する予測を実行
    for entry_dic in tqdm(ds_preprocessed[split].iter(batch_size=batch_size), total=len(ds_preprocessed[split])//batch_size+1):
        x, y = entry_dic["pixel_values"].to(device), entry_dic["labels"]
        output = model.forward(x, tgt_pos=tgt_pos, output_hidden_states_before_layernorm=False, output_intermediate_states=True)
        before_intermediate, intermediate, after_intermediate = output.intermediate_states[tgt_layer] # 3つの状態がタプルで入ってる
        proba = torch.nn.functional.softmax(output.logits, dim=-1).cpu().detach().numpy()
        # 予測結果のラベルを取得
        pred_labels = proba.argmax(axis=-1)
        all_proba.append(proba)
        all_pred_labels.append(pred_labels)
        all_before_intermediates.append(before_intermediate)
        all_intermediates.append(intermediate)
        all_after_intermediates.append(after_intermediate)
    all_proba = np.concatenate(all_proba)
    all_pred_labels = np.concatenate(all_pred_labels)
    all_before_intermediates = np.concatenate(all_before_intermediates)
    all_intermediates = np.concatenate(all_intermediates)
    all_after_intermediates = np.concatenate(all_after_intermediates)
    true_labels = np.array(ds[split][label_col])
    print(f"all_proba.shape: {all_proba.shape}, all_pred_labels.shape: {all_pred_labels.shape}, true_labels.shape: {true_labels.shape}")

Process repair (batch_size=32)...


  0%|          | 0/313 [00:00<?, ?it/s]

all_proba.shape: (10000, 100), all_pred_labels.shape: (10000,), true_labels.shape: (10000,)
Process test (batch_size=32)...


  0%|          | 0/313 [00:00<?, ?it/s]

all_proba.shape: (10000, 100), all_pred_labels.shape: (10000,), true_labels.shape: (10000,)


In [7]:
print(all_before_intermediates.shape)
print(all_intermediates.shape)
print(all_after_intermediates.shape)

(10000, 768)
(10000, 3072)
(10000, 768)
